# Ce notebook classe des phases en 3 catégories "cher", "moyen","éco".
C'est le premier d'une série de notebook pour comparer différentes techniques d'encodeing de textes et de classification.'

Keras uses tensorflow uses sklearn uses numpy and pandas

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder


Using TensorFlow backend.


 DATA PROCESS
 les données ont été saisies sous excel et sauvegardées en csv utf-8 bizarement avec l'option séparateur virgules de microsoft
 


In [2]:
def getDataset():
	dataset = pd.read_csv("RuleCout2.csv", delimiter=";", encoding='utf-8')
	#print ( 'dataset loaded with shape',dataset.shape)
	X = dataset['text']
	Y = dataset['class']
	dataset.set_index('text') #pour avoir les classes dans un ordre quelconque
	return dataset, X, Y

# utile pour les colonnes sur une ligne et afficher toutes les lignes
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 1000)

# on ajoutera peu à peu de nouvelles colonnes au dataset 
dataset, X, Y = getDataset()
print(dataset)

                         text  class
0     aucun problème d'argent   cher
1                         bon   cher
2                       cher    cher
3              gastronomique    cher
4              haut de gamme    cher
5            j’ai les moyens    cher
6              je m’en fiche    cher
7               je m’en fous    cher
8              je suis riche    cher
9   le meilleur des meilleurs   cher
10                      luxe    cher
11                   luxueux    cher
12      ma bourse est pleine    cher
13       moins de cent euros    cher
14             plus de vingt    cher
15                plutôt bien   cher
16                plutôt cher   cher
17                plutôt cher   cher
18             plutôt pas mal   cher
19                    qualité   cher
20                  standing    cher
21            super standing    cher
22         très haut de gamme   cher
23         très haut de gamme   cher
24              un bon repas    cher
25           un plat à vingt    cher
2

#### Les mots les plus signifiants ont été classés par catégorie.

La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "troquet" indique des classes de restaurant
La catégorie "négation" indique des mots qui inverse le sens
La catégorie "éco" indique les mots qui vont dans le sens de choix économiques
La catégorie "monnaie" indique des quantités monayables
La catégorie "ambigu" contient des termes qui se caractérisent par leur absence de positionnenement qui est une information en soit
Les catégories "nombres" sont explicites.

Les mots les plus signifiants vont permettre un encdage par catégorie.
ansi "pas trop cher" deviendra "négation 0 cher" ou [5 0 10]. 
On se posera la question de la pérsence du zéro : est-elle bénéfique ou néfaste ?

#### ce travail sur les ensembles impacte fortement le taux de réussite



In [3]:
eco = ['modique','économique','ténu','japonais', 'chinois','kebab', 'self','pourri','avantageux','marché','bas']
troquet = ['troquet','bistrot','cantine','self']
negation = ['pas','ni','peu']
cher = ['cher','bon','haut','luxe','luxueux','fiche','fous','moyens', 'gastronomique', 'classe','riche','michelin','etoile','étoiles',
'standing','haut','chicos','onéreux','onereux','super''meilleur','meilleurs','grattin','gratin']
monnaie = ['gamme', 'euro', 'prix', 'euros','problème','pb','qualité']
ambigu = ['milieu','moyen']
nombreEco = ['0','5','10','15','zéro','cinq','dix','quinze']
nombreCher= ['20','30','40','50','80','100','cent','vingt','trente','quarante','cinquante','quatre-vingt','cent']

# le choix de l'entier n'importe pas sauf pour le zéro qui est la valeur complémentée par pad_sequences
def score(word):
	if word in eco: 
		return 2
	elif word in nombreEco:
		return 3
	elif word in troquet: 
		return 4
	elif word in negation:
		return 5
	elif word in monnaie:
		return 6
	elif word in ambigu:
		return 7
	elif word in nombreCher:
		return 9
	elif word in cher:
		return 10
	else: 
		return 0
	
print ( 'score modique', score('modique'))
print ( 'score troquet', score('troquet'))
print ( 'score inconnu', score('inconnu'))


score modique 2
score troquet 4
score inconnu 0


In [4]:
#ai préféré utiliser le tokenizer de keras plutôt que celui de nltk car les valeurs filtrées par défaut comprennent l'apostrophe
def tokenize(sentence):
	tokens = text_to_word_sequence(sentence)
	return tokens
	
dataset['tokens']= dataset['text'].map(tokenize)
dataset['tokens']

0        [aucun, problème, d'argent]
1                              [bon]
2                             [cher]
3                    [gastronomique]
4                  [haut, de, gamme]
5                [j’ai, les, moyens]
6                  [je, m’en, fiche]
7                   [je, m’en, fous]
8                  [je, suis, riche]
9     [le, meilleur, des, meilleurs]
10                            [luxe]
11                         [luxueux]
12         [ma, bourse, est, pleine]
13          [moins, de, cent, euros]
14                 [plus, de, vingt]
15                    [plutôt, bien]
16                    [plutôt, cher]
17                    [plutôt, cher]
18                [plutôt, pas, mal]
19                         [qualité]
20                        [standing]
21                 [super, standing]
22           [très, haut, de, gamme]
23           [très, haut, de, gamme]
24                  [un, bon, repas]
25              [un, plat, à, vingt]
26         [un, restau, de, qualité]
2

L'étape suivante consiste à transformer une liste de tokens en une liste de nombres à encoder.
Comme la taille de cette liste doit être fixe on utilise pad_sequence qui ajoute des zéros ou enlève des nombres. 

In [5]:
def token2code(tokens):
	return list(map(score,tokens))

#dataset['Xcodes'] = list (map(token2code,dataset['tokens']))
def encode(data):
	dataset['Xcodes']= dataset['tokens'].map(token2code)
	return data
		
dataset = encode(dataset)
XEncoded = pad_sequences(dataset['Xcodes'])
dataset

,text,class,tokens,Xcodes
0,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0, 6, 0]"
1,bon,cher,[bon],[10]
2,cher,cher,[cher],[10]
3,gastronomique,cher,[gastronomique],[10]
4,haut de gamme,cher,"[haut, de, gamme]","[10, 0, 6]"
5,j’ai les moyens,cher,"[j’ai, les, moyens]","[0, 0, 10]"
6,je m’en fiche,cher,"[je, m’en, fiche]","[0, 0, 10]"
7,je m’en fous,cher,"[je, m’en, fous]","[0, 0, 10]"
8,je suis riche,cher,"[je, suis, riche]","[0, 0, 10]"
9,le meilleur des meilleurs,cher,"[le, meilleur, des, meilleurs]","[0, 0, 0, 10]"


Le prochain pavé permet de transformer les catégories "éco","moyen" et cher en vecteur.
Ces vecteurs sont rangés dans la colonne 'Ycodes' du dataset

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dataset['Ycodes'] = pd.Series(list(dummy_y))
dataset


,text,class,tokens,Xcodes,Ycodes
0,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0, 6, 0]","[1.0, 0.0, 0.0]"
1,bon,cher,[bon],[10],"[1.0, 0.0, 0.0]"
2,cher,cher,[cher],[10],"[1.0, 0.0, 0.0]"
3,gastronomique,cher,[gastronomique],[10],"[1.0, 0.0, 0.0]"
4,haut de gamme,cher,"[haut, de, gamme]","[10, 0, 6]","[1.0, 0.0, 0.0]"
5,j’ai les moyens,cher,"[j’ai, les, moyens]","[0, 0, 10]","[1.0, 0.0, 0.0]"
6,je m’en fiche,cher,"[je, m’en, fiche]","[0, 0, 10]","[1.0, 0.0, 0.0]"
7,je m’en fous,cher,"[je, m’en, fous]","[0, 0, 10]","[1.0, 0.0, 0.0]"
8,je suis riche,cher,"[je, suis, riche]","[0, 0, 10]","[1.0, 0.0, 0.0]"
9,le meilleur des meilleurs,cher,"[le, meilleur, des, meilleurs]","[0, 0, 0, 10]","[1.0, 0.0, 0.0]"


###MODEL DEFINITION
Le réseau comprend trois couches.
La première couche doit correspondre à la taille du vecteur X i.e au nombre de dimensions ici c'est cinq.
La couche de sortie comprend 3 neurons avec une fonction d'activation softmax, cette répartition entre les trois poids indiquent une probabilité d'appartenir à une des trois classes.
La taille de couche du milieu est ici de 11 avec une fonction d'activation Relu.


In [7]:
########################### MODEL DEFINITION
def sequential_model():
	# create model
	model = Sequential()
	model.add(Dense(11, input_dim=5, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model



In [9]:
#################### MODEL EVALUATION
model = sequential_model()
estimator = KerasClassifier(build_fn=sequential_model, epochs=200, batch_size=5, verbose=0)
	
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

dataset['predicted'] = list(model.predict(XEncoded)) 

results = cross_val_score(estimator, XEncoded, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 78.10% (13.50%)


In [11]:
def decodeY(L):
	if L[0]>L[1] and L[0]>L[2]: 
		return "cher"
	elif L[2]>L[1] and L[2]>L[0]:
		return "éco"
	else:
		return "moyen"
	
dataset['resultat']=list(map(decodeY,dataset['predicted']))
dataset


,text,class,tokens,Xcodes,Ycodes,predicted,resultat
0,aucun problème d'argent,cher,"[aucun, problème, d'argent]","[0, 6, 0]","[1.0, 0.0, 0.0]","[0.014642513, 0.7888279, 0.19652964]",moyen
1,bon,cher,[bon],[10],"[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
2,cher,cher,[cher],[10],"[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
3,gastronomique,cher,[gastronomique],[10],"[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
4,haut de gamme,cher,"[haut, de, gamme]","[10, 0, 6]","[1.0, 0.0, 0.0]","[0.0034656608, 0.8962653, 0.100269005]",moyen
5,j’ai les moyens,cher,"[j’ai, les, moyens]","[0, 0, 10]","[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
6,je m’en fiche,cher,"[je, m’en, fiche]","[0, 0, 10]","[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
7,je m’en fous,cher,"[je, m’en, fous]","[0, 0, 10]","[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
8,je suis riche,cher,"[je, suis, riche]","[0, 0, 10]","[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
9,le meilleur des meilleurs,cher,"[le, meilleur, des, meilleurs]","[0, 0, 0, 10]","[1.0, 0.0, 0.0]","[0.00417849, 0.84246624, 0.15335529]",moyen
